In [39]:
import numpy as np
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, MultiPolygon

d1 = np.array([[0,0], [0,1],  [1,0], [1,1]])
d2 = d1 + 0.5
d3 = d2 + 0.5
h1 = ConvexHull(d1)
h2 = ConvexHull(d2)
h3 = ConvexHull(d3)
v1 = [d1[v] for v in h1.vertices]
v2 = [d2[v] for v in h2.vertices]
v3 = [d3[v] for v in h3.vertices]
s1 = Polygon(v1)
s2 = Polygon(v2)
s3 = Polygon(v3)
polygons = [s1, s2, s3]
print reduce(lambda a,b: a.union(b), polygons[1:], polygons[0]).area

2.5


In [ ]:
def clusterGeometryMetrics(drum_points, labels):
    
    grouped_drum_points = {}
    
    for i in range(len(drum_points)):
        if lables[i] not in grouped_drum_points:
            group_drum_points[labels[i]] = set()
        group_drum_points[labels[i]].add(drum_points[i])
    
    num_labels = len(grouped_drum_points)
    drum_polygons = {}
    
    for drum in grouped_drum_points:
        points = grouped_drum_points[drum]
        hull = ConvexHull(grouped_drum_points[drum])
        drum_polygons[drum] = Polygon([points[v] for v in hull.vertices])
    
    total_hull = ConvexHull(drum_points)
    total_polygon = Polygon([drum_points[v] for v in total_hull.vertices])
    
    intersect = lambda i, j: drum_polygons[i].intersection(drum_polygons[j]).area / total_polygon.area
    
    pairwise_intersect_area = [[intersect(i,j) for i in range(num_lables)] for j in range(num_labels)]
    roundness = [4 * np.pi * drum_polygons[d].area / (drum_polygons[d].length**2) for d in drum_polygons]
    relative_area = [drum_polygons[d].area / total_polygon.area for d in drum_polygons]
    
    polygons = drum_polygons.values
    overlap_ratio = reduce(lambda a,b: a.union(b), polygons[1:], polygons[0]).area / sum([p.area for p in polygons])

In [32]:
sum(sum(np.array([[1, 2], [1, 2]])))

6